#Exercises

##1. Robot range finder. 
Measures 0m - 3m. Assuming distances are evently distributed, if faultly always measures < 1m. Prior probability of faultly sensor is p=0.01. If the sensor is meausered N times, and the value is always below 1m, what is the posterior probability that the sensor is faulty?

p(b|m) = p(m|b)*p(b)/P(m)


In [ ]:

prior = 0.01

for i in xrange(1,11):
    p_m = 1.0/3.0 * (1-prior) + 1*prior
    p_m_if_bad = 1.0
    p_bad = prior
    p_bad_if_m = p_m_if_bad * p_bad / p_m
    
    print('N=%d: %f%%' % (i, p_bad_if_m*100))
    prior = p_bad_if_m
    


## 2. Weather simulation

In [ ]:
#(a) day 1 = sunny. p day 2 = cloudy, day 3 = cloudy, day 4 = rainy?

p_2c = 0.2
p_3c = p_2c*0.4
p_4r = p_3c*0.3

print('probablity of sequence = %f' % p_4r)

In [ ]:
#(b)
import random
def getTomorrowsWeather(todaysWeather):
    r = random.random()
    if todaysWeather == 'sunny':
        if r < 0.8:
            return 'sunny'
        else:
            return 'cloudy'
    if todaysWeather == 'cloudy':
        if r < 0.4:
            return 'sunny'
        if r < 0.8:
            return 'cloudy'
        else:
            return 'rainy'
    if todaysWeather == 'rainy':
        if r < 0.2:
            return 'sunny'
        if r < 0.8:
            return 'cloudy'
        else:
            return 'rainy'
    print('Invalid weather!')
    return None

weather = 'sunny'
for i in xrange(10):
    print('N=%d: %s' % (i, weather))
    weather = getTomorrowsWeather(weather)

In [ ]:
#(c)
results = {}
results['sunny'] = 0
results['cloudy'] = 0
results['rainy'] = 0

weather = 'sunny'
N = 100000
for i in xrange(N):
    weather = getTomorrowsWeather(weather)
    results[weather] += 1

print('p(sunny) = %f%%' % (results['sunny']*100.0/N))
print('p(cloudy) = %f%%' % (results['cloudy']*100.0/N))
print('p(rainy) = %f%%' % (results['rainy']*100.0/N))


In [ ]:
import numpy as np

#(d) closed form solution

# S = (.8S + .4C + .2R), 0 = -.2S + .4C + .2 R
# C = (.2S + .4C + .6R), 0 = .2S + -.6C + .6R
# 1.0 = S + C + R

a = np.array([[-0.2, 0.4, 0.2],
              [0.2, -0.6, 0.6],
              [1.0, 1.0, 1.0]])
b = np.array([0.0,0.0,1.0])

p_sunny, p_cloudy, p_rainy = np.linalg.solve(a,b)
print('p(sunny) = %f%%' % (p_sunny*100.0))
print('p(cloudy) = %f%%' % (p_cloudy*100.0))
print('p(rainy) = %f%%' % (p_rainy*100.0))

#(e) Entropy
#entropy = -∑p(x)*log2(p(x))

ps = [p_sunny, p_cloudy, p_rainy]
ent = -reduce(lambda x, y: x+y, map(lambda p: p*np.log2(p), ps))
print ''
print ('entropy = %f' % ent)


## 3. Noisy Sensor

In [ ]:
# [sunny, cloudy, rainy]
p_w_0 = np.array([1.0,0.0,0.0])

def weather_update(w):
    """Returns the probabiliy of tomorrows weather given the probablity to todays weather."""
    t = np.array([[0.8, 0.2, 0.0],
                  [0.4, 0.4, 0.2],
                  [0.2, 0.6, 0.2]])
    return np.dot(w,t)

def normalize(p):
    """Normailzes the probablities to they add up to One"""
    return p / p.sum()

def measure_update(w, m):
    """Returns the probabiliy of todays weather given the prior probablity to todays weather and a measurement."""
    t = np.array([[0.6, 0.4, 0.0],
                  [0.3, 0.7, 0.0],
                  [0.0, 0.0, 1.0]]).T
    
    raw = w*t[m]
    return raw / raw.sum()


In [ ]:
ms = [1,1,2,0]
w = p_w_0[:]
for i, m in enumerate(ms):
    print 'day %d, m = %d' % (i + 2, m)
    w = weather_update(w)
    print '\t prior = %s' % str(w)
    w = measure_update(w,m)
    print '\t post = %s' % str(w)


## 4. Continuous Estimation

In [ ]:
def gauss(mu, sigma_sq, x):
    return (2*np.pi*sigma_sq)**-0.5 *np.exp(-(x-mu)**2/(2*sigma_sq))

gauss(0,np.sqrt(0.1),0.1)